In [9]:
import numpy as np
from scipy.linalg import null_space

In [10]:
def create_bootstrap_matrix(contract_ranges):
    m = len(contract_ranges)
    n = max([t[1] for t in contract_ranges])
    A = np.zeros((m, n))
    for i, contract in enumerate(contract_ranges):
        start_col, end_col = contract
        num_cols = end_col - start_col
        weight = 1.0/num_cols
        for j in range(start_col, end_col):
            A[i, j] = weight
    return A

In [11]:
def calc_bootstrap_example(contracts, flat_price):
    A = create_bootstrap_matrix(contracts)
    print('A')
    print(A)
    prices = np.array((flat_price, flat_price))
    print('prices')
    print(prices)
    pinv = np.linalg.pinv(A)
    print('pseudo-inverse')
    print(pinv)
    bootstrapped_prices = np.matmul(pinv, prices)
    print('bootstrapped')
    print(bootstrapped_prices)
    print('average of bootstrapped prices')
    print(np.mean(bootstrapped_prices))
    print('Length of bootstrapped prices')
    print(np.dot(bootstrapped_prices, bootstrapped_prices))
    print('Length of flat price vector')
    flat_price_array = np.full(bootstrapped_prices.shape, flat_price)
    print(np.dot(flat_price_array, flat_price_array))

In [12]:
# Badly Behaved Example
contracts = [
    (0, 3),
    (2, 4),
]
flat_price = 10
print('Badly Behaved Example')
calc_bootstrap_example(contracts, flat_price)

Badly Behaved Example
A
[[0.33333333 0.33333333 0.33333333 0.        ]
 [0.         0.         0.5        0.5       ]]
prices
[10 10]
pseudo-inverse
[[ 1.2 -0.4]
 [ 1.2 -0.4]
 [ 0.6  0.8]
 [-0.6  1.2]]
bootstrapped
[ 8.  8. 14.  6.]
average of bootstrapped prices
9.0
Length of bootstrapped prices
359.99999999999994
Length of flat price vector
400


In [13]:
# Well Behaved Example
contracts = [
    (0, 4),
    (2, 4),
]
flat_price = 10.0
print('Badly Behaved Example')
calc_bootstrap_example(contracts, flat_price)

Badly Behaved Example
A
[[0.25 0.25 0.25 0.25]
 [0.   0.   0.5  0.5 ]]
prices
[10. 10.]
pseudo-inverse
[[ 2.00000000e+00 -1.00000000e+00]
 [ 2.00000000e+00 -1.00000000e+00]
 [-4.33100169e-16  1.00000000e+00]
 [-4.33100169e-16  1.00000000e+00]]
bootstrapped
[10. 10. 10. 10.]
average of bootstrapped prices
10.0
Length of bootstrapped prices
400.00000000000006
Length of flat price vector
400.0


In [14]:
def calc_bootstrap_example_with_nullspace(contracts, flat_price):
    A = create_bootstrap_matrix(contracts)
    print('A')
    print(A)
    prices = np.array((flat_price, flat_price))
    print('prices')
    print(prices)
    pinv = np.linalg.pinv(A)
    print('pseudo-inverse')
    print(pinv)
    ns = null_space(A)
    print('nullspace basis')
    print(ns)
    ns_proj_mat = np.matmul(ns, ns.T)
    print('nullspace project matrix')
    print(ns_proj_mat)
    target = np.array([[flat_price], [flat_price], [flat_price], [flat_price]])
    print('target')
    print(target)
    target_proj_into_ns = np.matmul(ns_proj_mat, target)
    print('target projected into nullspace')
    print(target_proj_into_ns)
    bootstrapped_prices = np.matmul(pinv, prices) + target_proj_into_ns.T
    print('bootstrapped')
    print(bootstrapped_prices)
    print('average of bootstrapped prices')
    print(np.mean(bootstrapped_prices))
    print('Length of bootstrapped prices')
    print(np.dot(bootstrapped_prices, bootstrapped_prices))
    print('Length of flat price vector')
    flat_price_array = np.full(bootstrapped_prices.shape, flat_price)
    print(np.dot(flat_price_array, flat_price_array))

In [15]:
# Badly Behaved Example With Nullspace Component
contracts = [
    (0, 3),
    (2, 4),
]
flat_price = 10
print('Badly Behaved Example With Nullspace Component')
calc_bootstrap_example_with_nullspace(contracts, flat_price)

Badly Behaved Example With Nullspace Component
A
[[0.33333333 0.33333333 0.33333333 0.        ]
 [0.         0.         0.5        0.5       ]]
prices
[10 10]
pseudo-inverse
[[ 1.2 -0.4]
 [ 1.2 -0.4]
 [ 0.6  0.8]
 [-0.6  1.2]]
nullspace basis
[[-0.74489571 -0.21243911]
 [ 0.33825438  0.69683856]
 [ 0.40664134 -0.48439945]
 [-0.40664134  0.48439945]]
nullspace project matrix
[[ 0.6 -0.4 -0.2  0.2]
 [-0.4  0.6 -0.2  0.2]
 [-0.2 -0.2  0.4 -0.4]
 [ 0.2  0.2 -0.4  0.4]]
target
[[10]
 [10]
 [10]
 [10]]
target projected into nullspace
[[ 2.]
 [ 2.]
 [-4.]
 [ 4.]]
bootstrapped
[[10. 10. 10. 10.]]
average of bootstrapped prices
10.0
Length of bootstrapped prices


ValueError: shapes (1,4) and (1,4) not aligned: 4 (dim 1) != 1 (dim 0)

In [ ]:
svd = np.linalg.svd(A)
svd

In [ ]:
ns = null_space(A)
ns

In [ ]:
target = np.array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0])
print('target')
print(target)
error = target - bootstrapped_prices
print('error')
print(error)


In [ ]:
c = np.matmul(ns.T, error)
print(c)

In [ ]:
y = bootstrapped_prices + np.matmul(ns, c)
y

https://github.com/cmdty/curves/issues/8

In [ ]:
import datetime
from curves import bootstrap_contracts
contracts = [# Day contracts
 (datetime.date(2020, 4, 3), datetime.date(2020, 4, 3), 18.02),
 (datetime.date(2020, 4, 6), datetime.date(2020, 4, 6), 12.73),
 # Week contracts
 (datetime.date(2020, 4, 6), datetime.date(2020, 4, 12), 16.0),
 (datetime.date(2020, 4, 13), datetime.date(2020, 4, 19), 16.78),
 (datetime.date(2020, 4, 20), datetime.date(2020, 4, 26), 20.82),
 (datetime.date(2020, 4, 27), datetime.date(2020, 5, 3), 17.97), # This is what is causing issues
 # Month contracts
 (datetime.date(2020, 5, 1), datetime.date(2020, 5, 31), 22.21),
 (datetime.date(2020, 6, 1), datetime.date(2020, 6, 30), 27.3)]

piecewise_curve, bootstrapped_contracts = bootstrap_contracts(contracts, freq='D')
print(piecewise_curve['2020-04-25':'2020-05-05'])

In [ ]:
print(bootstrapped_contracts)

---
https://github.com/cmdty/curves/issues/12


In [ ]:
test_contracts = [(datetime.date(2020, 8, 24), datetime.date(2020, 8, 30), 10.0), 
(datetime.date(2020, 8, 31), datetime.date(2020, 9, 6), 10.0),
(datetime.date(2020, 9, 1), datetime.date(2020, 9, 30), 10.0)]

test_piecewise_curve, test_bootstrapped_contracts = bootstrap_contracts(test_contracts, freq='D')
print(test_piecewise_curve)

In [16]:
import pandas as pd

date_str = '2020-01-01'
freq='H'

period = pd.Period(date_str, freq=freq)
print(period)
ts = pd.Timestamp(date_str, tz='Europe/London')
print(ts)

offset = pd.tseries.frequencies.to_offset(freq)

print(period - offset)
print(ts - offset)

C:\Users\fowle\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


2020-01-01 00:00
2020-01-01 00:00:00+00:00
2019-12-31 23:00
2019-12-31 23:00:00+00:00


In [17]:
from datetime import date, datetime
pd.Timestamp(pd.Timestamp(2023,1, 1))

Timestamp('2023-01-01 00:00:00')

In [18]:
freq='30min'
offset = pd.tseries.frequencies.to_offset(freq)
period1 = pd.Period('2020-03-08', freq=freq)
period2 = pd.Period('2020-03-09', freq=freq)
freq_offset = pd.tseries.frequencies.to_offset(freq)
print(type(freq_offset))
round((period2 - period1)/freq_offset)

<class 'pandas._libs.tslibs.offsets.Minute'>


48

In [55]:
ts1 = pd.Timestamp('2020-03-08')
ts2 = pd.Timestamp('2020-06-02')

td = (ts2 - ts1)
print(td)

td2 = td.ceil(freq)
print(td2)
td2.to_numpy()

print((ts2 - ts1)/freq_offset)

86 days 00:00:00
86 days 00:00:00
4128.0


In [71]:
ts_index_d = pd.date_range(start=ts1, end=ts2, freq='D')
ts_index_30m = pd.date_range(start=ts1, end=ts2, freq='30min')


td_index_d = ts_index_d - ts1
td_index_30m = ts_index_30m - ts1

#print(td_index_d.days)
#td_index_30m.plot()
print(type(td_index_30m[-2]))
print((td_index_30m.total_seconds().to_numpy()))

#print(td_index_30m.total_seconds())


<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
[0.0000e+00 1.8000e+03 3.6000e+03 ... 7.4268e+06 7.4286e+06 7.4304e+06]


In [69]:
period_index_d = pd.period_range(start=ts1, end=ts2, freq='D')
period_index_30m = pd.period_range(start=ts1, end=ts2, freq='30min')

period_index_d = period_index_d - period_index_d[0]
period_index_30m = period_index_30m - period_index_30m[0]

print(type(period_index_d[1]))
print(period_index_30m)


<class 'pandas._libs.tslibs.offsets.Day'>
Index([      <0 * Minutes>,     <900 * Minutes>,    <1800 * Minutes>,
          <2700 * Minutes>,    <3600 * Minutes>,    <4500 * Minutes>,
          <5400 * Minutes>,    <6300 * Minutes>,    <7200 * Minutes>,
          <8100 * Minutes>,
       ...
       <3707100 * Minutes>, <3708000 * Minutes>, <3708900 * Minutes>,
       <3709800 * Minutes>, <3710700 * Minutes>, <3711600 * Minutes>,
       <3712500 * Minutes>, <3713400 * Minutes>, <3714300 * Minutes>,
       <3715200 * Minutes>],
      dtype='object', length=4129)
